# Bayesian inference of the Light Curve PSD

In this tutorial, we will demonstrate how you can use MCMC with `pgmuvi` to get a posterior distribution of the PSD of a light curve. We will use the same type of data as in the [basic tutorial](pgmuvi_tutorial.ipynb).

## Why Bayesian inference?


## Why MCMC?




## This tutorial

This tutorial will cover the following topics:

* How to set up a model for Bayesian inference of the PSD of a light curve
* How to run MCMC
* How to visualize the results

It unfortunately cannot tell you everything there is to know about fitting timeseries data with MCMC using `pgmuvi`, but it aims to give you a good starting point for your own projects.

### Some imports

Before we do anything, we need to make sure that `pgmuvi` imports correctly, and if it doesn't we need to install it. We also import some other packages that we will need later.

In [ ]:
try: #This won't work right now - instead clone the repository and `pip install -e .`
    import pgmuvi
except ImportError:
    %pip install git+https://github.com/ICSM/pgmuvi.git
    import pgmuvi

In [ ]:
#torch.multiprocessing.get_all_sharing_strategies()

### Creating the data

Now that we have imported `pgmuvi`, we can create some data to fit. 
We will use the same type of data as in the [basic tutorial](pgmuvi_tutorial.ipynb), but we will use a different random seed to get different data. 
This data is drawn from a sine wave with a randomly-chosen period between 30 and 300 days, and a Gaussian noise component with a standard deviation of 0.1 times the absolute value of the flux. The times are randomly chosen to cover between 3 and 10 periods of the sine wave, with 40 points in total.

In [ ]:
import torch
torch.multiprocessing.set_sharing_strategy('file_system')
import gpytorch
import numpy as np
""" Let's generate some synthetic data from a perturbed sine curve
    but on the same time sampling as the real data"""

P = np.random.uniform(30, 300)#137. #Days!
print("True period: ",P," days")
n_data = 40
jd_min = 2450000
n_periods = np.random.uniform(3,10)
jd_max = jd_min + P*(n_periods)
print("Simulating for ",n_periods," periods")

#train_mag =
#train_mag = train_mag + 0.1*torch.randn_like(train_mag)
#train_mag_err = 0.1*train_mag

period_guess = P*(np.random.uniform()+0.5)#147 #this number is in the same units as our original input.

#generate data from a simple case - superimpose two sine curves and add noise
timestamps_1d = torch.sort(torch.Tensor(np.random.uniform(jd_min, jd_max, size=n_data)))[0]#generate random x data here
fluxes_1d = torch.sin(timestamps_1d*(2*np.pi/P))#generate random y data here
fluxes_1d += 0.1*torch.randn_like(fluxes_1d)
flux_err_1d = 0.1*fluxes_1d.abs()
print("Generated data with ",n_data," points")
print("Period guess: ",period_guess," days")
print("Period guess: ",period_guess/P," periods")
print(timestamps_1d)
print(fluxes_1d)
print(flux_err_1d)

### Getting the Lightcurve object

In [ ]:
from pgmuvi.lightcurve import Lightcurve

lightcurve_1d = Lightcurve(timestamps_1d, fluxes_1d, yerr = flux_err_1d, xtransform='minmax')

### Our Model

Now we can create our model. 
This is very similar to the previous tutorial, but with one small complication. 
When we didn't use MCMC, we wanted to learn additional diagonal noise to account for the intrinsic scatter of the data even if we had uncertainties on the data. 
However, when we use MCMC, we need to be careful about how we define our likelihood.
If we attempt to learn this additional noise, `gpytorch` will inject `NaN`s along the diagonal of the covariance matrix, which will cause the MCMC sampler to fail. 
However, if you don't have uncertainty information, you can still learn this additional noise.

In [ ]:
# This won't work! Learning the additional noise results in NaNs in the covariance matrix during MCMC
# lightcurve_1d.set_model(model='1D', likelihood='learn', num_mixtures=1)

In [ ]:
lightcurve_1d.set_model(model='1D', num_mixtures=1)

In [ ]:
lightcurve_1d.print_parameters()
print(period_guess)
guess = {
         'sci_kernel.mixture_means': torch.Tensor([1/period_guess]),}
lightcurve_1d.set_hypers(guess)
lightcurve_1d.print_parameters()

In [ ]:
print(lightcurve_1d._xdata_transformed)
print(lightcurve_1d._ydata_transformed)
print(lightcurve_1d._yerr_transformed)

### Doing MCMC

In [ ]:
#try:
lightcurve_1d.mcmc(num_samples=100, #0, 
                   warmup_steps=100, 
                   num_chains=1)#num_samples=1000, burnin=100, thin=10, num_chains=4, num_steps=10, num_processes=4)
#except Exception as e:
    #print(e)
    #lightcurve_1d.mcmc(num_samples=100, num_chains=2)#num_samples=1000, burnin=100, thin=10, num_chains=4, num_steps=10, num_processes=4)

### Interpreting the results

First we want to make a quick summary of the results.
We can do this by plotting the marginal posterior distributions of each parameter, and by printing out the mean and standard deviation of each parameter. 
The mean and standard deviation are useful because they give us a sense of the "best fit" parameters, and the uncertainty on those parameters - or more formally, the credible interval of the parameters and the point estimate.

In [ ]:
lightcurve_1d.summary()

We also want to take a look at how the MCMC sampler explored the parameter space. 
This is often referred to as the "trace" of the MCMC sampler.

In [ ]:
lightcurve_1d.plot_trace()

In [ ]:
lightcurve_1d.plot_corner()

Plotting the marginal posterior distributions is easy with `plot_pair`. `pgmuvi` often works best with `kind='scatter'`, but you can also try out `kind='kde'` or `kind='hexbin'` for prettier plots.

We also want to plot the light curve with samples from the posterior distribution of the model. 
This lets us see how well the model fits the data, and how well the model can predict the data.

In [ ]:
# plot goes here
lightcurve_1d.plot(mcmc_samples=True)

We might also want to look at the posterior distribution of the PSD. 
This is useful to understand how much the data constrain the PSD, and hence how much we can trust the results. 
The same information is effectively contained in the posterior distribution of the model parameters (as reported in the summary or displayed in the trace and the corner plot), but it is often easier to understand the PSD directly.

In [ ]:
# plot goes here
lightcurve_1d.plot_psd(mcmc_samples=True, log=(True, True))